In [1]:
import json
import os
import itertools
from typing import Tuple

import pandas as pd
import numpy as np
import talib as ta
from tqdm import tqdm

from ftx_client import FtxClient
import supertrend as spt
import backtesting as bt
from config import API_KEY, API_SECRET

In [2]:
with open("settings.json") as jsonfile:
    settings = json.load(jsonfile)
    
BACKTEST_FOLDER = settings["filepaths"]["backtest_folder"]
OPTIMIZEDML_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["optimized_ml_file"])
ANALYSIS_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["analysis_file"])

In [3]:
ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

In [4]:
markets = []
for future in ftx.list_futures():
    if future["type"] == "perpetual":
        if (future["volumeUsd24h"] > settings["markets"]["min_volume_usd_24h"] and 
                future["name"] not in settings["markets"]["blacklist"]):
            markets.append(future["name"])

In [5]:
optimzed_ml = pd.read_csv(OPTIMIZEDML_FILEPATH)

In [6]:
analysis_df = pd.DataFrame(columns=["Name", "AvgReturns", "StdDev", "AvgDrawdown", "RetDevRatio", "MinReturns", "MaxReturns",
             "AvgNegReturns", "AvgPosReturns", "PosNegRetRatio", "MedReturns", "MedNegReturns",
             "MedPosReturns", "MedPosNegRetRatio", "TheDfactor"])

pbar = tqdm(markets)
for market in pbar:
    pbar.set_description(market)
    df = ftx.get_historical_market_data(market, interval=settings["analysis"]["interval"], start_time=settings["analysis"]["start_time"])
    
    if len(df) < settings["analysis"]["min_data_length"]:
        continue
        
    multiplier = optimzed_ml.loc[optimzed_ml['Name'] == market]["Multiplier"].values[0]
    lookback = optimzed_ml.loc[optimzed_ml['Name'] == market]["Lookback"].values[0]
    
    profits = bt.backtest_dataframe(df, look_back=lookback, multiplier=multiplier)
    profits["Name"] = market
    analysis_df = analysis_df.append(profits, ignore_index=True)

XRP-PERP: 100%|██████████| 47/47 [02:28<00:00,  3.15s/it]  


In [7]:
analysis_df = analysis_df.round(1)
analysis_df.to_csv(ANALYSIS_FILEPATH, index=False)

In [8]:
pd.read_csv(ANALYSIS_FILEPATH).sort_values("TheDfactor", ascending=False)

,Name,AvgReturns,StdDev,AvgDrawdown,RetDevRatio,MinReturns,MaxReturns,AvgNegReturns,AvgPosReturns,PosNegRetRatio,MedReturns,MedNegReturns,MedPosReturns,MedPosNegRetRatio,TheDfactor
15,DOGE-PERP,23.9,27.3,12.0,0.9,-12.4,62.3,-4.3,45.0,10.5,26.0,-0.3,45.9,139.2,11.6
9,BSV-PERP,46.4,62.9,2.0,0.7,4.5,171.4,0.0,46.4,46.4,18.4,0.0,18.4,18.4,9.1
26,LTC-PERP,18.3,33.0,4.3,0.6,-4.6,113.2,-1.7,35.0,20.3,1.5,-0.8,23.1,28.4,6.5
7,BCH-PERP,28.8,34.1,3.5,0.8,-1.7,108.7,-1.7,32.6,18.9,12.9,-1.7,17.8,10.3,3.0
18,EOS-PERP,25.0,28.1,3.5,0.9,-3.1,77.6,-1.9,32.7,16.9,9.7,-1.9,19.3,10.0,2.9
29,MID-PERP,31.6,26.8,8.5,1.2,-2.4,63.2,-2.4,40.1,16.5,32.5,-2.4,45.8,18.8,2.2
4,ALPHA-PERP,65.2,115.8,5.8,0.6,-2.2,344.8,-2.2,76.4,35.0,26.9,-2.2,27.0,12.4,2.1
28,MATIC-PERP,60.9,112.3,13.2,0.5,-5.4,321.9,-2.7,108.7,40.2,2.2,-2.1,55.3,26.7,2.0
12,CHZ-PERP,17.0,25.8,5.5,0.7,-2.8,94.4,-1.5,21.1,13.7,8.2,-1.5,14.4,9.4,1.7
17,ENJ-PERP,19.3,32.4,7.2,0.6,-2.9,94.1,-1.7,35.1,20.6,4.1,-1.9,21.0,10.9,1.5
